In [2]:
import os
import sys

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

In [3]:
from bs4 import BeautifulSoup
from collections import defaultdict
from flow_wmd.documents import Document
from flow_wmd.models import LC_RWMD, WMD, WMDManyToMany
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cluster

import numpy as np
import pandas as pd
import re

%load_ext autoreload
%autoreload 2

/Users/mikaelbrunila/Documents/Code/flow-wmd/venv-fwmd/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## 1. Prepare IMDB data

### 1.1 Load data and stopwords.

In [4]:
PATH = "../data/"
imdb_data = pd.read_csv(f"{PATH}IMDB_Dataset.csv")
stopword_list=stopwords.words('english')

### 1.2 Initialize cleanup functions 

Functions to remove noisy formatting, lemmatizing and removing stopwords.

In [5]:
# Custom preprocessing functions
# Partly self-authored, partly from https://www.kaggle.com/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews

#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square <brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = re.sub('<br / ><br / >', ' ', text)
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z\s]'
    text=re.sub(pattern,'',text)
    return text

#Lemmatizing the text
def simple_lemmatizer(text):
    lemmatizer=WordNetLemmatizer() 
    text= ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

#removing the stopwords
def remove_stopwords(text, stopword_list, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token.lower() for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

### 1.3 Remove special formatting and stopwords

Remove stopwords before denoising, lemmatizing and removing special characters.

In [6]:
%time 

tokenizer=ToktokTokenizer()
imdb_data['review_clean']= [remove_stopwords(r, stopword_list) for r in imdb_data['review']]

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.05 µs


Denoise, remove special characters, lemmatize.

In [7]:
%time

imdb_data['review_clean']=imdb_data['review_clean'].apply(denoise_text)
imdb_data['review_clean']=imdb_data['review_clean'].apply(remove_special_characters)
imdb_data['review_clean']=imdb_data['review_clean'].apply(simple_lemmatizer)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


Remove stopwords again, after other preprocessing.

In [8]:
%time 

imdb_data['review_clean']= [remove_stopwords(r, stopword_list) for r in imdb_data['review_clean']]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


Data _before_ preprocessing.

In [9]:
imdb_data['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

Data _after_ preprocessing.

In [10]:
imdb_data['review_clean'][0]

'one reviewer mentioned watching oz episode hooked right exactly happened first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experience watching oz m

### 1.4 Separate pos and neg reviews

In [11]:
pos = imdb_data[imdb_data.sentiment == "positive"].reset_index(drop=True)
neg = imdb_data[imdb_data.sentiment == "negative"].reset_index(drop=True)

In [12]:
pos = pos.review_clean.tolist()
neg = neg.review_clean.tolist()

## 2. WMD

### 2.1 Tokenize and "sample" data

In [13]:
def tokenize(text):
    tokens = tokenizer.tokenize(text)
    return tokens

pos_tok = list(map(tokenize, pos[:500]))
neg_tok = list(map(tokenize, neg[:500]))

In [14]:
pos_sample = [" ".join(doc) for doc in pos_tok]
neg_sample = [" ".join(doc) for doc in neg_tok]

### 2.2 Load pretrained Google News W2V model

In [19]:
def read_1w_corpus(name, sep="\t"):
    for line in open(name):
        yield line.split(sep)

print("Loading GoogleNews Vectors")
%time model = KeyedVectors.load_word2vec_format('../embeddings/GoogleNews-vectors-negative300.bin.gz', binary=True)

Loading GoogleNews Vectors
CPU times: user 44.5 s, sys: 2.77 s, total: 47.3 s
Wall time: 48.2 s


### 2.3 Load corpus and remove OOV words

In [20]:
corpus = pos_sample + neg_sample

%time vectorizer = TfidfVectorizer(use_idf=False, tokenizer=tokenize, norm='l1')
%time vectorizer.fit(corpus)

CPU times: user 35 µs, sys: 20 µs, total: 55 µs
Wall time: 59.1 µs


/Users/mikaelbrunila/Documents/Code/flow-wmd/venv-fwmd/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CPU times: user 313 ms, sys: 908 ms, total: 1.22 s
Wall time: 2.53 s


TfidfVectorizer(norm='l1', tokenizer=<function tokenize at 0x7fb18310bca0>,
                use_idf=False)

In [24]:
%time oov = [word for word in vectorizer.get_feature_names() if word not in model.key_to_index.keys()]

CPU times: user 17.3 ms, sys: 574 µs, total: 17.9 ms
Wall time: 17.6 ms


In [25]:
len(oov)

3066

In [26]:
#removing the oov words
def remove_oov(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in oov]
    #filtered_tokens = filter(lambda token: token not in oov, tokens)
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

%time pos_sample = list(map(remove_oov, pos_sample))
%time neg_sample = list(map(remove_oov, neg_sample))

CPU times: user 2.3 s, sys: 9.27 ms, total: 2.31 s
Wall time: 2.31 s
CPU times: user 2.18 s, sys: 8.79 ms, total: 2.18 s
Wall time: 2.19 s


In [27]:
pos_sample[0]

'one reviewer mentioned watching oz episode hooked right exactly happened first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word called oz nickname given maximum security state focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experience watching oz may become comfortab

In [28]:
corpus = pos_sample + neg_sample

%time vectorizer = TfidfVectorizer(use_idf=True, tokenizer=tokenize,norm='l1')
%time vectorizer.fit(corpus)

CPU times: user 23 µs, sys: 0 ns, total: 23 µs
Wall time: 25 µs
CPU times: user 219 ms, sys: 5.64 ms, total: 224 ms
Wall time: 230 ms


TfidfVectorizer(norm='l1', tokenizer=<function tokenize at 0x7fb18310bca0>)

Bag-of-words vectorizer.

In [29]:
%time
pos_nbow = vectorizer.transform(pos_sample)
neg_nbow = vectorizer.transform(neg_sample)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [30]:
pos_tok = list(map(tokenize, pos_sample))
neg_tok =  list(map(tokenize, neg_sample))

In [31]:
pos_tok[0][:20]

['one',
 'reviewer',
 'mentioned',
 'watching',
 'oz',
 'episode',
 'hooked',
 'right',
 'exactly',
 'happened',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scene',
 'violence',
 'set',
 'right']

In [34]:
%time oov_ = [word for word in vectorizer.get_feature_names() if word not in model.key_to_index.keys()]

CPU times: user 14 ms, sys: 223 µs, total: 14.2 ms
Wall time: 14.3 ms


In [35]:
len(oov_)

0

### 2.4 Get features and embeddings

In [36]:
features = vectorizer.get_feature_names()
word2idx = {word: idx for idx, word in enumerate(vectorizer.get_feature_names())}
idx2word = {idx: word for idx, word in enumerate(vectorizer.get_feature_names())}

Get the embedding matrix "E" for all features.

In [38]:
E = np.vstack([model.get_vector(word) for word in vectorizer.get_feature_names()])

### Cluster

In [41]:
X = model[model.index_to_key]

In [46]:
NUM_CLUSTERS = 100
kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
kmeans.fit(X)

KMeans(n_clusters=100)

In [50]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [87]:
word2cluster = {model.index_to_key[idx]: cl for idx, cl in enumerate(labels)}

In [91]:
cluster2words = defaultdict(list)
for key, value in word2cluster.items():
    cluster2words[value].append(key)

In [92]:
print(cluster2words[0][:100])

['Mr', 'Australia', 'Government', 'Australian', 'New_Zealand', 'Sydney', 'Labor', 'Melbourne', 'Ms', 'Victoria', 'Queensland', 'NSW', 'Liberal', 'Brisbane', 'Opposition', 'Auckland', 'Wellington', 'Australians', 'Crown', 'Perth', 'Liberals', 'Fraser', 'petrol', 'Victorian', 'Adelaide', 'NZ', 'Fiji', 'Aussie', 'AFL', 'Telstra', 'Canberra', 'Fairfax', 'Christchurch', 'Greens', 'Rudd', 'Canterbury', 'ACT', 'Swan', 'Gold_Coast', 'Qantas', 'WA', 'Murdoch', 'Bali', 'Macquarie', 'Darwin', 'Carlton', 'Aboriginal', 'bush', 'Harbour', 'Kiwi', 'Tasmania', 'Plenty', 'NAB', 'Coles', 'ANZ', 'Maori', 'Papua', 'cyclone', 'premiership', 'RBA', 'Geelong', 'Hobart', 'breaching', 'Latham', 'Packer', 'asylum_seekers', 'Cairns', 'ASX', 'Beattie', 'Liberal_Party', 'Mackay', 'Cousins', 'New_Zealanders', 'Southland', 'East_Timor', 'QC', 'Mr_Rudd', 'Woolworths', 'Dunedin', 'aboriginal', 'Westpac', 'AAP', 'Graeme', 'GST', 'Napier', 'Northland', 'Anglican', 'Otago', 'Woodside', 'AMP', 'APEC', 'Territory', 'Cr', '

### 2.5 Initialize documents

Transform all reviews into "documents", each with a set of weights per word in the corpus ("nbow"), the sum of these weights ("weights_sum"), the indeces of the words in the documents ("idxs") and the word vectors corresponding to each word ("vecs").

In [102]:
%time 

pos_docs, neg_docs = [], []

for idx, doc in enumerate(pos_tok):
    pos_docs.append(Document(doc, pos_nbow[idx], word2idx, E))
    
for idx, doc in enumerate(neg_tok):
    neg_docs.append(Document(doc, neg_nbow[idx], word2idx, E))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs


In [105]:
pos_docs[0].nbow

array([[0., 0., 0., ..., 0., 0., 0.]])

In [106]:
pos_docs[0].weights_sum

1.0

In [108]:
pos_docs[0].idxs[:10]

[12299, 9740, 1036, 2071, 10266, 5148, 13342, 13344, 11302, 5673]

In [116]:
pos_docs[0].vecs[:1][0][:10]

array([ 0.16992188,  0.04907227,  0.08154297,  0.12011719, -0.14746094,
        0.0291748 ,  0.36523438, -0.10107422,  0.125     ,  0.04516602],
      dtype=float32)

### 2.6 Linear-Complexity Relaxed WMD (LC-RWMD)

Run the [Linear-Complexity Relaxed WMD](https://arxiv.org/abs/1711.07227) to get the distances between all positive and all negative reviews.

In [119]:
%time lc_rwmd = LC_RWMD(pos_docs, neg_docs,pos_nbow,neg_nbow,E)
%time lc_rwmd.get_D()
#%time lc_rwmd.get_L(1)
#%time lc_rwmd.get_rwmd()

CPU times: user 15 µs, sys: 51 µs, total: 66 µs
Wall time: 67.7 µs
CPU times: user 2min 20s, sys: 25.6 s, total: 2min 46s
Wall time: 43.7 s


### 2.7 Gale-Shapeley Pairing

Use the [Gale-Shapeley matching algorithm](https://en.wikipedia.org/wiki/Gale%E2%80%93Shapley_algorithm) to find the optimal pairs between positive and negative reviews. This iterates over all the reviews and finds the set of matches that pairs each review with its optimal match given that all positive reviews have to be matched with a negative review and vice versa. The output is a dictionary of key-value pairs, where each pair represents an optimal match.

In [120]:
from flow_wmd.gale_shapeley import Matcher

matcher = Matcher(lc_rwmd.D)
engaged = matcher.matchmaker()
matcher.check()
pairs = engaged

Let's look at the output of Gale-Shapeley:

In [128]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))


take(10, pairs.items())

[(137, 497),
 (388, 468),
 (95, 360),
 (454, 361),
 (259, 363),
 (263, 146),
 (143, 90),
 (224, 107),
 (91, 108),
 (443, 347)]

### 2.8 Pairwise WMD

Calculate the pairwise distances between the documents selected by the Galey-Shapeley algorithm _without_ returning the flow between individual words.

In [135]:
from flow_wmd.models import WMDPairs

wmd_pairs = WMDPairs(pos_docs,neg_docs,pairs,E,idx2word)
%time wmd_pairwise = wmd_pairs.get_distances()

Calculated distances between 0 documents.
Calculated distances between 100 documents.
Calculated distances between 200 documents.
Calculated distances between 300 documents.
Calculated distances between 400 documents.
CPU times: user 4min 18s, sys: 16.5 s, total: 4min 34s
Wall time: 3min 7s


The return value is a matrix of distances between the document pairs.

In [136]:
wmd_pairwise

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Calculate the pairwise distances between the documents selected by the Galey-Shapeley algorithm, this time also returning the flow between individual words.

In [137]:
wmd_pairs_flow = WMDPairs(pos_docs,neg_docs,pairs,E,idx2word)
%time wmd_pairwise_flow = wmd_pairs_flow.get_distances(return_flow = True)

Calculated distances between 0 documents.
Calculated distances between 100 documents.
Calculated distances between 200 documents.
Calculated distances between 300 documents.
Calculated distances between 400 documents.
CPU times: user 4min 23s, sys: 17.7 s, total: 4min 41s
Wall time: 3min 11s


Now we have three return values.

The first one is again a matrix of distances between the document pairs.

In [141]:
wmd_pairwise_flow[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

The second return value is a list of tuples with all the words that contributed the most to the distance from the positive documents to the negative ones. These are _not_ sorted from high to low or vice versa.

In [142]:
take(10, wmd_pairwise_flow[1].items())

[('alleyway', 0.02905),
 ('claude', 0.04238),
 ('buddhist', 0.04406),
 ('slays', 0.02581),
 ('barrier', 0.05491),
 ('hark', 0.0319),
 ('performing', 0.35353999999999997),
 ('lola', 0.03157),
 ('headmistress', 0.01871),
 ('goofball', 0.09673000000000001)]

The third return value is a list of tuples with all the words that contributed the most to the distance from the negative documents to the positive ones. Again, these are _not_ sorted from high to low or vice versa.

In [144]:
take(10, wmd_pairwise_flow[2].items())

[('barrier', 0.07387),
 ('performing', 0.14185),
 ('lola', 0.02067),
 ('variable', 0.01497),
 ('gambling', 0.0281),
 ('flaw', 0.11897),
 ('hitting', 0.04962),
 ('versatile', 0.023600000000000003),
 ('heaven', 0.18339),
 ('hellraiser', 0.02917)]

### 2.9 Intepreting pairwise WMD flows

Now, let's sort the distances of the words that created the most distance from the positive to the negative reviews.

In [149]:
{k: v for k, v in sorted(wmd_pairwise_flow[1].items(), key=lambda item: item[1], reverse=True)[:30]}

{'film': 4.534129999999999,
 'movie': 3.938469999999998,
 'story': 3.434129999999999,
 'great': 3.077940000000002,
 'love': 2.9721399999999996,
 'performance': 2.73945,
 'life': 2.426700000000001,
 'character': 2.39385,
 'best': 2.340230000000001,
 'well': 2.3356999999999988,
 'watch': 2.2191299999999994,
 'show': 2.1959499999999994,
 'still': 2.195579999999999,
 'scene': 2.19522,
 'good': 2.1382,
 'time': 2.09075,
 'comedy': 2.08384,
 'young': 2.0808300000000015,
 'one': 2.0784700000000003,
 'like': 2.015480000000001,
 'dvd': 2.01107,
 'excellent': 2.0090300000000005,
 'people': 1.9920699999999996,
 'fan': 1.9595799999999997,
 'actor': 1.87092,
 'role': 1.86876,
 'see': 1.8347,
 'little': 1.8331200000000005,
 'loved': 1.83208,
 'family': 1.7825199999999999}

Next, let's see what added most distance when moving from the negative to the positive reviews.

In [151]:
{k: v for k, v in sorted(wmd_pairwise_flow[2].items(), key=lambda item: item[1], reverse=True)[:30]}

{'movie': 6.2731,
 'bad': 5.4052500000000006,
 'film': 4.1417399999999995,
 'worst': 3.635940000000001,
 'plot': 3.55549,
 'waste': 3.1293699999999993,
 'scene': 3.013090000000001,
 'acting': 2.7281000000000004,
 'character': 2.5816599999999994,
 'made': 2.4689400000000004,
 'rating': 2.45743,
 'like': 2.45132,
 'ever': 2.3756300000000006,
 'stupid': 2.34381,
 'actor': 2.2481300000000006,
 'story': 2.22648,
 'funny': 2.2264,
 'really': 2.2117,
 'boring': 2.2083199999999996,
 'watch': 2.19631,
 'good': 2.1373299999999995,
 'even': 2.0962,
 'thing': 2.0919399999999997,
 'money': 2.0855600000000005,
 'watching': 2.039080000000001,
 'guy': 1.9936299999999998,
 'awful': 1.9861900000000003,
 'would': 1.9547700000000006,
 'seen': 1.9501299999999995,
 'see': 1.93109}

## Appendix: Many-to-many WMD

This was a first attempt to do the flows from words between many documents, without first filtering using Gale-Shapeley. However, this proved too inefficient. As you can see looking at the CPU times, it is very slow even with extremely small samples and the time complexity is quadratic (or worse?), meaning it rapidly gets even worse as the sample size increases.

In [155]:
%time m2m_distances = WMDManyToMany(pos_docs[:20], neg_docs[:20],E,idx2word).get_distances(return_flow = False)

CPU times: user 1min 48s, sys: 12.7 s, total: 2min 1s
Wall time: 50 s


In [156]:
%time m2m_distances_flow, wc_X1, wc_X2 = WMDManyToMany(pos_docs[:20],neg_docs[:20],E,idx2word).get_distances(return_flow = True)

CPU times: user 1min 52s, sys: 13.5 s, total: 2min 5s
Wall time: 51.4 s


In [157]:
{k: v for k, v in sorted(wc_X1.items(), key=lambda item: item[1], reverse=True)[:10]}

{'karen': 8.69223,
 'wrenching': 8.31882,
 'carpenter': 7.468960000000001,
 'laughter': 7.467879999999999,
 'liked': 6.864090000000003,
 'mom': 6.791519999999999,
 'gut': 6.759419999999999,
 'love': 6.551409999999997,
 'camp': 6.533080000000001,
 'hr': 6.1393699999999995}

In [158]:
{k: v for k, v in sorted(wc_X2.items(), key=lambda item: item[1], reverse=True)[:10]}

{'hopper': 8.372459999999998,
 'jake': 7.63837,
 'movie': 7.267059999999995,
 'film': 6.936379999999998,
 'shakespeare': 5.99276,
 'oddness': 5.53033,
 'terrible': 4.943440000000001,
 'parent': 4.751790000000001,
 'actor': 4.672620000000001,
 'bad': 4.430020000000002}